In [1]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [2]:
# Set base path
base_path = os.getcwd()

In [3]:
# Load script from specified path sys.argv[1] (snakemake {input})
# Does nothing else than `import path/to/script as exp` would do
experiment_file = './experiments/experiment_stabilized_groundstate.py'
conf_name, _ = os.path.splitext(os.path.basename(experiment_file))
conf_path = os.path.join(os.getcwd(), './experiments/experiment_stabilized_groundstate.py')
spec = importlib.util.spec_from_file_location(conf_name, conf_path)
exp = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp)

In [4]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
outpath = net_params['outpath']

# Set scaling parameters
net_params['N_scaling'] = 0.001
net_params['K_scaling'] = 0.001

### Creating the network

Here the network is created using the `NeuronNumbers` and `SynapseNumbers` classes. The `NeuronNumbers` class calculates the number of neurons in the network. The `SynapseNumbers` class calculates the number of synapses in the network and the synaptic weights.

In [5]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

# Create Network
net = Network(NN, SN, net_params)

# Export the network
net.dump(outpath)

# Get the network hash
net_hash = net.getHash()

# To access net object key, use net.net.keys()
# Ex.: to access neuron numbers, use net.net['neuron_numbers']

# Save the network hash
hash_file = net_hash
hash_fn = os.path.join(os.getcwd(), hash_file)
with open(hash_fn, 'w') as f:
    f.write(net.getHash())

# Save the humam repository hash
net_folder = os.path.join(outpath, net.getHash())
git_fn = os.path.join(net_folder, "git_hash.txt")
with open(git_fn, 'w') as f:
    f.write(get_git_revision_hash())

Dropping layer IV in caudalmiddlefrontal because it contains 3375 < 5000 neurons.
Dropping layer IV in lateraloccipital because it contains 3000 < 5000 neurons.
Dropping layer VI in parahippocampal because it contains 3200 < 5000 neurons.
Dropping layer IV in parsorbitalis because it contains 625 < 5000 neurons.


### Running the simulation

In [6]:
from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_sim_params import params as sim_params
from network import networkDictFromDump
from simulation import Simulation

[INFO] [2024.5.2 16:15:48 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:217 @ Network::create_rngs_] : Creating default RNGs
[INFO] [2024.5.2 16:15:48 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:260 @ Network::create_grng_] : Creating new default global RNG

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: nest-2.20.2
 Built: Nov 15 2021 06:23:02

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [7]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
nested_dict_update(sim_params, exp.sim_params)

In [8]:
# Read network dict
net_folder = os.path.join(outpath, net_hash)
net_dict = networkDictFromDump(net_folder)

In [9]:
# Create Simulation class, export it and calculate the hash
sim = Simulation(sim_params, net_dict)
sim.dump(net_folder)
sim_hash = sim.getHash()

In [10]:
# Set output directory according to hashes, instantiate the Network
# and run the simulation
data_path = os.path.join(outpath, net_hash, sim_hash)
num_threads = int(4)
sim.setup(data_path, num_threads)
sim.simulate()

data directory already exists
Data will be written to /home/shimoura/Documents/GitHub/humam/out/d9815dd7185e65c5602f790ee0f7026c/de4934b8c7777751f7c516e2ad35f50a

May 02 16:15:48 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

May 02 16:15:48 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

May 02 16:15:48 Network::create_rngs_ [Info]: 
    Creating default RNGs

May 02 16:15:48 Network::create_grng_ [Info]: 
    Creating new default global RNG
Master seed: 2106 

May 02 16:15:48 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

May 02 16:15:48 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

May 02 16:15:48 Network::create_rngs_ [Info]: 
    Creating default RNGs

May 02 16:15:48 Network::create_grng_ [Info]: 
    Creating new default global RNG
Number of total processes: 4
Seeds for random number generators of virtual processes: [2111, 2112, 2113, 21

NESTErrors.ArgumentType: ('ArgumentType in Create: The type of one or more parameters did not match the argument(s) of this function.', 'ArgumentType', <SLILiteral: Create>, ': The type of one or more parameters did not match the argument(s) of this function.')

In [ ]:
# Save the simulation hash
hash_fn = os.path.join(os.getcwd(), sim_hash)
with open(hash_fn, 'w') as f:
    f.write(sim_hash)

# Save the humam repository hash
sim_folder = os.path.join(net_folder, sim_hash)
git_fn = os.path.join(sim_folder, "git_hash.txt")
with open(git_fn, 'w') as f:
    f.write(get_git_revision_hash())

### Analysis

In [ ]:
from network import networkDictFromDump
from simulation import simulationDictFromDump
from analysis import Analysis
from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_ana_params import params as ana_params
from default_net_params import params as net_params

In [ ]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
nested_dict_update(ana_params, exp.ana_params)

# Read simulation dict
sim_folder = os.path.join(outpath, net_hash, sim_hash)
sim_dict = simulationDictFromDump(sim_folder)

# Create Analysis class and export it
ana = Analysis(ana_params, net_dict, sim_dict, sim_folder, base_path)
ana.dump(sim_folder)

# Do the analysis
ana.fullAnalysis()

In [ ]:
ana.plotRasterArea('bankssts')